<a href="https://colab.research.google.com/github/vinbaskaran/AI_projects/blob/main/ShopAssistAI_2.0_Project_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ShopAssist AI - 2.0 - Enhanced Approach using Function Calling

Dataset obtained from : https://drive.google.com/file/d/1Z4Qta-ZpYGc-jjFstJXckyJKIvoQXTJj/view?usp=sharing

Starter Notebook obtained from: https://colab.research.google.com/drive/1T-a5WIcle8XhBS5aX5tzFbc2S3MwFnT1#scrollTo=ad9b47b8



# Starter Notebook

In [ ]:
# Import the libraries
import pandas as pd
from IPython.display import display, HTML
# Set the display width to control the output width
pd.set_option('display.width', 100)

In [ ]:
# Import the libraries
import os, json, ast
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt
from google.colab import userdata

In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"

    example_user_dict = {'GPU intensity': "high",
                        'Display quality':"high",
                        'Portability': "low",
                        'Multitasking': "high",
                        'Processing speed': "high",
                        'Budget': "150000"}

    example_user_req = {'GPU intensity': "_",
                        'Display quality': "_",
                        'Portability': "_",
                        'Multitasking': "_",
                        'Processing speed': "_",
                        'Budget': "_"}

    system_message = f"""
    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.
    You need to ask relevant questions and understand the user profile by analysing the user's responses.
    You final objective is to fill the values for the different keys ('GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget') in the python dictionary and be confident of the values.
    These key value pairs define the user's profile.
    The python dictionary looks like this
    {{'GPU intensity': 'values','Display quality': 'values','Portability': 'values','Multitasking': 'values','Processing speed': 'values','Budget': 'values'}}
    The value for 'Budget' should be a numerical value extracted from the user's response.
    The values for all keys, except 'Budget', should be 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    All the values in the example dictionary are only representative values.
    {delimiter}
    Here are some instructions around the values for the different keys. If you do not follow this, you'll be heavily penalised:
    - The values for all keys, except 'Budget', should strictly be either 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    - The value for 'Budget' should be a numerical value extracted from the user's response.
    - 'Budget' value needs to be greater than or equal to 25000 INR. If the user says less than that, please mention that there are no laptops in that range.
    - Do not randomly assign values to any of the keys.
    - The values need to be inferred from the user's response.
    {delimiter}

    To fill the dictionary, you need to have the following chain of thoughts:
    Follow the chain-of-thoughts below and only output the final updated python dictionary for the keys as described in {example_user_req}. \n
    {delimiter}
    Thought 1: Ask a question to understand the user's profile and requirements. \n
    If their primary use for the laptop is unclear. Ask followup questions to understand their needs.
    You are trying to fill the values of all the keys {{'GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget'}} in the python dictionary by understanding the user requirements.
    Identify the keys for which you can fill the values confidently using the understanding. \n
    Remember the instructions around the values for the different keys.
    If the necessary information has been extracted, only then proceed to the next step. \n
    Otherwise, rephrase the question to capture their profile clearly. \n

    {delimiter}
    Thought 2: Now, you are trying to fill the values for the rest of the keys which you couldn't in the previous step.
    Remember the instructions around the values for the different keys.
    Ask questions you might have for all the keys to strengthen your understanding of the user's profile.
    If yes, move to the next Thought. If no, ask question on the keys whose values you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the key you want to understand value for.
    {delimiter}

    {delimiter}
    Thought 3: Check if you have correctly updated the values for the different keys in the python dictionary.
    If you are not confident about any of the values, ask clarifying questions.
    {delimiter}

    {delimiter}
    Here is a sample conversation between the user and assistant:
    User: "Hi, I am an editor."
    Assistant: "Great! As an editor, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high multitasking capability. You would also need a high end display for better visuals and editing. May I know what kind of work do you primarily focus on? Are you more involved in video editing, photo editing, or both? Understanding the specific type of editing work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now."
    User: "I primarily work with After Effects."
    Assistant: "Thank you for providing that information. Working with After Effects involves working with graphics, animations, and rendering, which will require high GPU. Do you work with high-resolution media files, such as 4K videos or RAW photos? Understanding your file sizes will help determine the storage capacity and processing power needed."
    User: "Yes, sometimes I work with 4K videos as well."
    Assistant: "Thank you for the information. Processing 4K vidoes will require a good processor and high GPU. I think we have already determined earlier that you need a high GPU. To ensure I have a complete understanding of your needs, I have one more question: Are you frequently on the go and require a laptop that is lightweight and easy to carry, or do you primarily work from a stationary location?"
    User: "Yes, sometimes I travel but do not carry my laptop."
    Assistant:"Could you kindly let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements."
    User: "my max budget is 1.5lakh inr"
    Assistant: "{example_user_dict}"
    {delimiter}

    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message}]
    # conversation = system_message
    return conversation

In [ ]:
def moderation_check(user_input):
    # Call the OpenAI API to perform moderation on the user's input.
    response = openai.moderations.create(input=user_input)

    #print(response)

    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if response.results[0].flagged == True:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"

In [ ]:
def intent_confirmation_layer(response_assistant):

    delimiter = "####"

    allowed_values = {'low','medium','high'}

    prompt = f"""
    You are a senior evaluator who has an eye for detail.The input text will contain a user requirement captured through 6 keys.
    You are provided an input. You need to evaluate if the input text has the following keys:
    {{
    'GPU intensity': 'values',
    'Display quality':'values',
    'Portability':'values',
    'Multitasking':'values',
    'Processing speed':'values',
    'Budget':'number'
    }}
    The values for the keys should only be from the allowed values: {allowed_values}.
    The 'Budget' key can take only a numerical value.
    Next you need to evaluate if the keys have the the values filled correctly.
    Only output a one-word string in JSON format at the key 'result' - Yes/No.
    Thought 1 - Output a string 'Yes' if the values are correctly filled for all keys, otherwise output 'No'.
    Thought 2 - If the answer is No, mention the reason in the key 'reason'.
    THought 3 - Think carefully before the answering.
    """

    messages=[{"role": "system", "content":prompt },
              {"role": "user", "content":f"""Here is the input: {response_assistant}""" }]

    response = openai.chat.completions.create(
                                    model="gpt-3.5-turbo",
                                    messages = messages,
                                    response_format={ "type": "json_object" },
                                    seed = 1234
                                    # n = 5
                                    )

    json_output = json.loads(response.choices[0].message.content)

    return json_output

In [ ]:
def dictionary_present(response):
    delimiter = "####"

    user_req = {'GPU intensity': 'high',
                'Display quality': 'high',
                'Portability': 'medium',
                'Multitasking': 'high',
                'Processing speed': 'high',
                'Budget': '200000'}

    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract the relevant values from the input and return only the python dictionary in JSON format.
            The output should match the format as {user_req}.

            {delimiter}
            Make sure that the value of budget is also present in the user input. ###
            The output should contain the exact keys and values as present in the input.
            Ensure the keys and values are in the given format:
            {{
            'GPU intensity': 'low/medium/high ',
            'Display quality':'low/medium/high',
            'Portability':'low/medium/high',
            'Multitasking':'low/medium/high',
            'Processing speed':'low/medium/high',
            'Budget':'numerical value'
            }}
            Here are some sample input output pairs for better understanding:
            {delimiter}
            input 1: - GPU intensity: low - Display quality: high - Portability: low - Multitasking: high - Processing speed: medium - Budget: 50,000 INR
            output 1: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '50000'}}

            input 2: {{'GPU intensity':     'low', 'Display quality':     'low', 'Portability':    'medium', 'Multitasking': 'medium', 'Processing speed': 'low', 'Budget': '90,000'}}
            output 2: {{'GPU intensity': 'low', 'Display quality': 'low', 'Portability': 'medium', 'Multitasking': 'medium', 'Processing speed': 'low', 'Budget': '90000'}}

            input 3: Here is your user profile 'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000 INR'
            output 3: {{'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000'}}
            {delimiter}
            """
    messages = [{"role": "system", "content":prompt },
                {"role": "user", "content":f"""Here is the user input: {response}""" }]

    confirmation = get_chat_completions(messages, json_format = True)

    return confirmation

In [ ]:
def product_map_layer(laptop_description):
    delimiter = "#####"

    lap_spec = {
        "GPU intensity":"(Type of the Graphics Processor)",
        "Display quality":"(Display Type, Screen Resolution, Display Size)",
        "Portability":"(Laptop Weight)",
        "Multitasking":"(RAM Size)",
        "Processing speed":"(CPU Type, Core, Clock Speed)"
    }

    values = {'low','medium','high'}

    prompt=f"""
    You are a Laptop Specifications Classifier whose job is to extract the key features of laptops and classify them as per their requirements.
    To analyze each laptop, perform the following steps:
    Step 1: Extract the laptop's primary features from the description {laptop_description}
    Step 2: Store the extracted features in {lap_spec} \
    Step 3: Classify each of the items in {lap_spec} into {values} based on the following rules: \
    {delimiter}
    GPU Intensity:
    - low: <<< if GPU is entry-level such as an integrated graphics processor or entry-level dedicated graphics like Intel UHD >>> , \n
    - medium: <<< if mid-range dedicated graphics like M1, AMD Radeon, Intel Iris >>> , \n
    - high: <<< high-end dedicated graphics like Nvidia RTX >>> , \n

    Display Quality:
    - low: <<< if resolution is below Full HD (e.g., 1366x768). >>> , \n
    - medium: <<< if Full HD resolution (1920x1080) or higher. >>> , \n
    - high: <<< if High-resolution display (e.g., 4K, Retina) with excellent color accuracy and features like HDR support. >>> \n

    Portability:
    - high: <<< if laptop weight is less than 1.51 kg >>> , \n
    - medium: <<< if laptop weight is between 1.51 kg and 2.51 kg >>> , \n
    - low: <<< if laptop weight is greater than 2.51 kg >>> \n

    Multitasking:
    - low: <<< If RAM size is 8 GB, 12 GB >>> , \n
    - medium: <<< if RAM size is 16 GB >>> , \n
    - high: <<< if RAM size is 32 GB, 64 GB >>> \n

    Processing Speed:
    - low: <<< if entry-level processors like Intel Core i3, AMD Ryzen 3 >>> , \n
    - medium: <<< if Mid-range processors like Intel Core i5, AMD Ryzen 5 >>> , \n
    - high: <<< if High-performance processors like Intel Core i7, AMD Ryzen 7 or higher >>> \n
    {delimiter}

    {delimiter}
    Here is input output pair for few-shot learning:
    input 1: "The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience. With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment. All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users."
    output 1: {{'GPU intensity': 'medium','Display quality':'medium','Portability':'medium','Multitasking':'high','Processing speed':'medium'}}

    {delimiter}
    ### Strictly don't keep any other text in the values of the JSON dictionary other than low or medium or high ###
    """
    input = f"""Follow the above instructions step-by-step and output the dictionary in JSON format {lap_spec} for the following laptop {laptop_description}."""
    #see that we are using the Completion endpoint and not the Chatcompletion endpoint
    messages=[{"role": "system", "content":prompt },{"role": "user","content":input}]

    response = get_chat_completions(messages, json_format = True)

    #   response = openai.chat.completions.create(
    #     model="gpt-3.5-turbo-0125",
    #     messages=[{"role": "system", "content":prompt },{"role": "user","content":input}],
    #     response_format={ "type": "json_object" }
    #     # max_tokens = 2000,
    #     )

    # response = json.loads(response)
    return response

In [ ]:
import pandas as pd  # Importing the pandas library for data manipulation
import json
import ast # Import the ast module

def compare_laptops_with_user(user_req_string):
    laptop_df = pd.read_csv('updated_laptop.csv')

    # Convert the 'laptop_feature' column from string representation of dictionary to actual dictionary
    laptop_df['laptop_feature'] = laptop_df['laptop_feature'].apply(ast.literal_eval)

    # Use the input user_req_string to get the user requirements dictionary
    user_requirements = dictionary_present(user_req_string)

    # Extracting the budget value from user_requirements and converting it to an integer
    budget_value = user_requirements.get('Budget', '0')
    if isinstance(budget_value, str):
        budget = int(budget_value.replace(',', '').split()[0])
    else:
        budget = int(budget_value)


    # Creating a copy of the DataFrame and filtering laptops based on the budget
    filtered_laptops = laptop_df.copy()
    filtered_laptops['Price'] = filtered_laptops['Price'].str.replace(',', '').astype(int)
    filtered_laptops = filtered_laptops[filtered_laptops['Price'] <= budget].copy()

    # Mapping string values 'low', 'medium', 'high' to numerical scores 0, 1, 2
    mappings = {'low': 0, 'medium': 1, 'high': 2}

    # Creating a new column 'Score' in the filtered DataFrame and initializing it to 0
    filtered_laptops['Score'] = 0

    # Iterating over each laptop in the filtered DataFrame to calculate scores based on user requirements
    for index, row in filtered_laptops.iterrows():
        laptop_values = row['laptop_feature'] # Now laptop_values is a dictionary

        score = 0

        # Comparing user requirements with laptop features and updating scores
        for key, user_value in user_requirements.items():
            if key == 'Budget':
                continue  # Skipping budget comparison
            laptop_value = laptop_values.get(key, None)
            laptop_mapping = mappings.get(laptop_value, -1)
            user_mapping = mappings.get(user_value, -1)
            if laptop_mapping >= user_mapping:
                score += 1  # Incrementing score if laptop value meets or exceeds user value

        filtered_laptops.loc[index, 'Score'] = score  # Updating the 'Score' column in the DataFrame

    # Sorting laptops by score in descending order and selecting the top 3 products
    top_laptops = filtered_laptops.drop('laptop_feature', axis=1)
    top_laptops = top_laptops.sort_values('Score', ascending=False).head(3)
    top_laptops_json = top_laptops.to_json(orient='records')  # Converting the top laptops DataFrame to JSON format


    return top_laptops_json

In [ ]:
def recommendation_validation(laptop_recommendation):
    data = json.loads(laptop_recommendation)
    data1 = []
    for i in range(len(data)):
        if data[i]['Score'] > 2:
            data1.append(data[i])

    return data1

In [ ]:
def initialize_conv_reco(products):
    system_message = f"""
    You are an intelligent laptop gadget expert and you are tasked with the objective to \
    solve the user queries about any product from the catalogue in the user message \
    You should keep the user profile in mind while answering the questions.\

    Start with a brief summary of each laptop in the following format, in decreasing order of price of laptops:
    1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>

    """
    user_message = f""" These are the user's products: {products}"""
    conversation = [{"role": "system", "content": system_message },
                    {"role":"user","content":user_message}]
    # conversation_final = conversation[0]['content']
    return conversation

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_chat_completions(input, json_format = False):
    MODEL = 'gpt-3.5-turbo'

    system_message_json_output = """<<. Return output in JSON format to the key output.>>"""

    # If the output is required to be in JSON format
    if json_format == True:
        # Append the input prompt to include JSON response as specified by OpenAI
        input[0]['content'] += system_message_json_output

        # JSON return type specified
        chat_completion_json = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            response_format = { "type": "json_object"},
            seed = 1234)

        output = json.loads(chat_completion_json.choices[0].message.content)

    # No JSON return type specified
    else:
        chat_completion = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            seed = 2345)

        output = chat_completion.choices[0].message.content

    return output

In [ ]:
def dialogue_mgmt_system():
    conversation = initialize_conversation()

    introduction = get_chat_completions(conversation)

    display(introduction)

    top_3_laptops = None

    user_input = ''

    while(user_input != "exit"):

        user_input = input("")

        moderation = moderation_check(user_input)
        if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break

        if top_3_laptops is None:

            conversation.append({"role": "user", "content": user_input})

            response_assistant = get_chat_completions(conversation)
            moderation = moderation_check(response_assistant)
            if moderation == 'Flagged':
                display("Sorry, this message has been flagged. Please restart your conversation.")
                break


            confirmation = intent_confirmation_layer(response_assistant)

            print("Intent Confirmation Yes/No:",confirmation.get('result'))

            if "No" in confirmation.get('result'):
                conversation.append({"role": "assistant", "content": str(response_assistant)})
                print("\n" + str(response_assistant) + "\n")

            else:
                print("\n" + str(response_assistant) + "\n")
                print('\n' + "Variables extracted!" + '\n')

                response = dictionary_present(response_assistant)

                print("Thank you for providing all the information. Kindly wait, while I fetch the products: \n")
                top_3_laptops = compare_laptops_with_user(response)

                print("top 3 laptops are", top_3_laptops)

                validated_reco = recommendation_validation(top_3_laptops)

                conversation_reco = initialize_conv_reco(validated_reco)

                conversation_reco.append({"role": "user", "content": "This is my user profile" + str(response)})

                recommendation = get_chat_completions(conversation_reco)

                moderation = moderation_check(recommendation)
                if moderation == 'Flagged':
                    display("Sorry, this message has been flagged. Please restart your conversation.")
                    break

                conversation_reco.append({"role": "assistant", "content": str(recommendation)})

                print(str(recommendation) + '\n')
        else:
            conversation_reco.append({"role": "user", "content": user_input})

            response_asst_reco = get_chat_completions(conversation_reco)

            moderation = moderation_check(response_asst_reco)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            print('\n' + response_asst_reco + '\n')
            conversation.append({"role": "assistant", "content": response_asst_reco})

In [ ]:
##Run this code once to extract product info in the form of a dictionary
laptop_df= pd.read_csv('laptop_data.csv')

## Create a new column "laptop_feature" that contains the dictionary of the product features
laptop_df['laptop_feature'] = laptop_df['Description'].apply(lambda x: product_map_layer(x))
laptop_df.to_csv("updated_laptop.csv",index=False,header = True)

In [57]:
dialogue_mgmt_system()

'Hello! Welcome to the laptop recommendation assistant. Please share with me your requirements so that I can help you find the best laptop that suits your needs.'

gaming
Intent Confirmation Yes/No: No

Great to know that you are into gaming! To recommend the best laptop for gaming, I need to understand your specific requirements. Are you more interested in high-quality graphics, smooth multitasking, portability, fast processing, or do you have a specific budget in mind? Understanding these aspects will help me tailor my recommendations to suit your needs.

high gpu, high processing speed, medium multitasking, medium display, low portability. Budget 125000
Intent Confirmation Yes/No: Yes

Based on your requirements, let's fill in the values for the keys in the dictionary:

{'GPU intensity': 'high', 'Display quality': 'medium', 'Portability': 'low', 'Multitasking': 'medium', 'Processing speed': 'high', 'Budget': 125000}


Variables extracted!

Thank you for providing all the information. Kindly wait, while I fetch the products: 

top 3 laptops are [{"Brand":"MSI","Model Name":"GL65","Core":"i7","CPU Manufacturer":"Intel","Clock Speed":"2.6 GHz","R

# Enhanced Function Calling Implementation

This section implements OpenAI function calling to replace the complex prompt-based approach for extracting user requirements and classifying laptop features.

## Identify functions that are suitable for function calling

Detailed analysis of all functions in the notebook, with an explanation of their suitability for the OpenAI function calling approach:

---

### 1. `initialize_conversation()`
**Purpose:**  
Creates the initial system prompt and conversation context for the assistant.

**Function Calling Suitability:**  
- **Not suitable** for OpenAI function calling.  
- **Reason:** This function is for system setup and prompt engineering, not for extracting structured data from user input or for being invoked by the LLM during a conversation.

---

### 2. `moderation_check(user_input)`
**Purpose:**  
Checks user input for unsafe or inappropriate content using OpenAI's moderation API.

**Function Calling Suitability:**  
- **Not suitable** for OpenAI function calling.  
- **Reason:** This is a backend safety check, not a user-facing function. It is not meant to be called by the LLM as part of the conversation flow, but rather as a guardrail in the application logic.

---

### 3. `intent_confirmation_layer(response_assistant)`
**Purpose:**  
Validates if the extracted user requirements dictionary is complete and correctly filled.

**Function Calling Suitability:**  
- **Not suitable** for OpenAI function calling.  
- **Reason:** This function is a validation/checking layer, not a user-facing function. It is used to ensure the LLM's output is correct, not to extract structured data from user input.

---

### 4. `dictionary_present(response)`
**Purpose:**  
Extracts a Python dictionary of user requirements from a string.

**Function Calling Suitability:**  
- **Suitable** for OpenAI function calling (with adaptation).  
- **Reason:** The core task—extracting structured data (user requirements) from free-form user input—is exactly what OpenAI function calling is designed for.  
- **How to adapt:**  
  - Define a function schema for the user requirements (e.g., keys for GPU intensity, display quality, etc.).
  - Let the LLM extract and return these as structured arguments.

---

### 5. `product_map_layer(laptop_description)`
**Purpose:**  
Extracts and classifies laptop features from a product description.

**Function Calling Suitability:**  
- **Suitable** for OpenAI function calling (with adaptation).  
- **Reason:** This function takes unstructured product descriptions and outputs structured feature data, which matches the function calling paradigm.  
- **How to adapt:**  
  - Define a function schema for laptop features (e.g., GPU intensity, display quality, etc.).
  - Let the LLM extract and return these as structured arguments.

---

### 6. `compare_laptops_with_user(user_req_string)`
**Purpose:**  
Compares user requirements with the laptop dataset and returns the top matches.

**Function Calling Suitability:**  
- **Not directly suitable** for OpenAI function calling.  
- **Reason:** This is a backend data processing function that operates on structured data and the local dataset. It is not meant to be invoked by the LLM to extract information from user input, but rather to process already-extracted data.

---

### 7. `recommendation_validation(laptop_recommendation)`
**Purpose:**  
Filters recommended laptops to ensure they meet a minimum score threshold.

**Function Calling Suitability:**  
- **Not suitable** for OpenAI function calling.  
- **Reason:** This is a backend validation/filtering function, not a user-facing function or one that extracts structured data from user input.

---

### 8. `initialize_conv_reco(products)`
**Purpose:**  
Creates a system prompt for the recommendation phase, including the recommended products.

**Function Calling Suitability:**  
- **Not suitable** for OpenAI function calling.  
- **Reason:** This is a system prompt construction function, not a user-facing function for extracting structured data.

---

### 9. `get_chat_completions(input, json_format=False)`
**Purpose:**  
Handles OpenAI chat completions, optionally requesting JSON output.

**Function Calling Suitability:**  
- **Not suitable** for OpenAI function calling.  
- **Reason:** This is a utility function for interacting with the OpenAI API, not a function to be called by the LLM.

---

### 10. `dialogue_mgmt_system()`
**Purpose:**  
Manages the overall conversation flow, calling other functions as needed.

**Function Calling Suitability:**  
- **Not suitable** for OpenAI function calling.  
- **Reason:** This is the main application logic, not a function to be invoked by the LLM.

---

## **Summary Table**

| Function Name                  | Suitable for Function Calling? | Reason                                                                 |
|------------------------------- |:-----------------------------:|------------------------------------------------------------------------|
| initialize_conversation        | No                            | System setup, not user-facing extraction                               |
| moderation_check               | No                            | Backend safety check                                                   |
| intent_confirmation_layer      | No                            | Validation/checking, not extraction                                    |
| dictionary_present             | Yes (with adaptation)         | Extracts structured user requirements from input                       |
| product_map_layer              | Yes (with adaptation)         | Extracts structured product features from description                  |
| compare_laptops_with_user      | No                            | Backend data processing, not user-facing extraction                    |
| recommendation_validation      | No                            | Backend validation/filtering                                           |
| initialize_conv_reco           | No                            | System prompt construction                                             |
| get_chat_completions           | No                            | Utility for API interaction                                            |
| dialogue_mgmt_system           | No                            | Main application logic                                                 |

---

## **Detailed Explanation**

- **Suitable Functions:**  
  - `dictionary_present` and `product_map_layer` are both suitable for OpenAI function calling because their main purpose is to extract structured data from unstructured input (user messages or product descriptions). This aligns perfectly with the function calling paradigm, where the LLM is asked to extract and return specific fields as arguments to a function.

- **Not Suitable Functions:**  
  - All other functions are either for system setup, backend processing, validation, or utility purposes. They do not extract structured data from user input or product descriptions, nor are they meant to be invoked by the LLM as part of the conversation flow.

- **How to Adapt:**  
  - For `dictionary_present`, define a function schema that matches the user requirements dictionary and let the LLM fill it via function calling.
  - For `product_map_layer`, define a function schema for product features and let the LLM extract these from product descriptions.

This approach will make the system more modular, robust, and efficient, leveraging OpenAI's function calling for the parts of the workflow that benefit most from structured data extraction.

## Implementation of Function Calling

**Function Calling Schemas**

- extract_user_requirements: Replaces the complex dictionary_present function

- classify_laptop_features: Replaces the complex product_map_layer function

Both use structured parameters with validation and clear descriptions

In [49]:
# Function calling schemas for OpenAI
function_descriptions = [
    {
        "name": "extract_user_requirements",
        "description": "Extract structured user laptop requirements from conversation. Use this when you have gathered enough information about the user's needs for GPU intensity, display quality, portability, multitasking, processing speed, and budget.",
        "parameters": {
            "type": "object",
            "properties": {
                "gpu_intensity": {
                    "type": "string",
                    "enum": ["low", "medium", "high"],
                    "description": "User's GPU requirements: low (integrated graphics), medium (mid-range dedicated), high (high-end dedicated like RTX)"
                },
                "display_quality": {
                    "type": "string",
                    "enum": ["low", "medium", "high"],
                    "description": "User's display requirements: low (below Full HD), medium (Full HD+), high (4K/Retina with color accuracy)"
                },
                "portability": {
                    "type": "string",
                    "enum": ["low", "medium", "high"],
                    "description": "User's portability needs: high (lightweight, travel), medium (moderate mobility), low (stationary use)"
                },
                "multitasking": {
                    "type": "string",
                    "enum": ["low", "medium", "high"],
                    "description": "User's multitasking requirements: low (basic tasks), medium (moderate workload), high (intensive multitasking)"
                },
                "processing_speed": {
                    "type": "string",
                    "enum": ["low", "medium", "high"],
                    "description": "User's processing speed needs: low (basic computing), medium (moderate performance), high (high-performance tasks)"
                },
                "budget": {
                    "type": "integer",
                    "minimum": 25000,
                    "description": "User's maximum budget in INR (minimum 25000)"
                }
            },
            "required": ["gpu_intensity", "display_quality", "portability", "multitasking", "processing_speed", "budget"]
        }
    },
    {
        "name": "classify_laptop_features",
        "description": "Extract and classify laptop specifications from product description into standardized categories",
        "parameters": {
            "type": "object",
            "properties": {
                "gpu_intensity": {
                    "type": "string",
                    "enum": ["low", "medium", "high"],
                    "description": "GPU classification: low (integrated/Intel UHD), medium (AMD Radeon/Intel Iris), high (Nvidia RTX/high-end)"
                },
                "display_quality": {
                    "type": "string",
                    "enum": ["low", "medium", "high"],
                    "description": "Display classification: low (<Full HD), medium (Full HD+), high (4K/Retina/HDR)"
                },
                "portability": {
                    "type": "string",
                    "enum": ["low", "medium", "high"],
                    "description": "Weight classification: high (<1.51kg), medium (1.51-2.51kg), low (>2.51kg)"
                },
                "multitasking": {
                    "type": "string",
                    "enum": ["low", "medium", "high"],
                    "description": "RAM classification: low (8-12GB), medium (16GB), high (32GB+)"
                },
                "processing_speed": {
                    "type": "string",
                    "enum": ["low", "medium", "high"],
                    "description": "CPU classification: low (i3/Ryzen 3), medium (i5/Ryzen 5), high (i7+/Ryzen 7+)"
                }
            },
            "required": ["gpu_intensity", "display_quality", "portability", "multitasking", "processing_speed"]
        }
    }
]


### **Enhanced Functions**

**Instead of `dictionary_present`:**
- ✅ `extract_user_requirements()` - Direct function calling with schema validation
- ✅ `initialize_conversation_with_function_calling()` - Simplified conversation flow

**Instead of complex `product_map_layer`:**  
- ✅ `classify_laptop_features()` - Direct classification function
- ✅ `product_map_layer_enhanced()` - Uses function calling for laptop feature extraction

### **Enhanced System Components**
- ✅ `get_chat_completions_with_functions()` - Supports function calling
- ✅ `dialogue_mgmt_system_enhanced()` - Complete enhanced dialogue system
- ✅ `compare_laptops_with_user_enhanced()` - Works with function calling output





In [50]:
def initialize_conversation_with_function_calling():
    """
    Enhanced conversation initialization that uses function calling for requirement extraction
    """
    system_message = """
    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.
    You need to ask relevant questions to understand the user's profile and requirements.

    Your conversation flow should be:
    1. Start with a brief welcome and ask about their primary laptop usage
    2. Ask follow-up questions to understand their specific needs (max 2 features per question)
    3. Once you have sufficient information about their GPU, display, portability, multitasking, processing, and budget needs, use the extract_user_requirements function

    Guidelines:
    - Don't ask about more than 2 features at a time
    - If budget is less than 25000 INR, mention no laptops available in that range
    - Use logical questions rather than directly asking about technical specifications
    - Only call extract_user_requirements when you're confident about all 6 parameters

    Example conversation flow:
    - Ask about primary use case (gaming, work, editing, etc.)
    - Ask about mobility needs and budget
    - Ask clarifying questions based on their responses
    - Extract requirements when complete

    Start with a short welcome message and encourage the user to share their requirements.
    """

    conversation = [{"role": "system", "content": system_message}]
    return conversation

In [51]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_chat_completions_with_functions(messages, use_functions=True):
    """
    Enhanced chat completion function that supports function calling
    """
    MODEL = 'gpt-4'  # Using GPT-4 for better function calling performance

    if use_functions:
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            functions=function_descriptions,
            function_call="auto",  # Let the model decide when to call functions
            temperature=0,
            seed=1234
        )
    else:
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            temperature=0,
            seed=1234
        )

    return response.choices[0].message

In [52]:
def extract_user_requirements(gpu_intensity, display_quality, portability, multitasking, processing_speed, budget):
    """
    Function called by OpenAI when user requirements are extracted
    This replaces the complex dictionary_present function
    """
    user_requirements = {
        'GPU intensity': gpu_intensity,
        'Display quality': display_quality,
        'Portability': portability,
        'Multitasking': multitasking,
        'Processing speed': processing_speed,
        'Budget': str(budget)
    }

    # Validation
    if budget < 25000:
        return {"error": "Budget too low - no laptops available below 25000 INR"}

    # Return structured requirements
    return user_requirements

In [53]:
def classify_laptop_features(gpu_intensity, display_quality, portability, multitasking, processing_speed):
    """
    Function called by OpenAI to classify laptop features from description
    This replaces the complex product_map_layer function
    """
    laptop_features = {
        'GPU intensity': gpu_intensity,
        'Display quality': display_quality,
        'Portability': portability,
        'Multitasking': multitasking,
        'Processing speed': processing_speed
    }

    return laptop_features

def product_map_layer_enhanced(laptop_description):
    """
    Enhanced version of product_map_layer using function calling
    """
    system_message = """
    You are a laptop specifications classifier. Analyze the given laptop description and classify its features.

    Use the classify_laptop_features function to extract and categorize:
    - GPU intensity: low (integrated/Intel UHD), medium (AMD Radeon/Intel Iris), high (Nvidia RTX)
    - Display quality: low (<Full HD), medium (Full HD+), high (4K/Retina/HDR)
    - Portability: high (<1.51kg), medium (1.51-2.51kg), low (>2.51kg)
    - Multitasking: low (8-12GB RAM), medium (16GB), high (32GB+)
    - Processing speed: low (i3/Ryzen 3), medium (i5/Ryzen 5), high (i7+/Ryzen 7+)

    Analyze the laptop description carefully and call the function with appropriate classifications.
    """

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": f"Classify this laptop: {laptop_description}"}
    ]

    response = get_chat_completions_with_functions(messages, use_functions=True)

    if response.function_call:
        function_name = response.function_call.name
        function_args = json.loads(response.function_call.arguments)

        if function_name == "classify_laptop_features":
            return classify_laptop_features(**function_args)

    # Fallback to original method if function calling fails
    return product_map_layer(laptop_description)

In [62]:
def dialogue_mgmt_system_enhanced():
    """
    Enhanced dialogue management system using function calling
    """
    conversation = initialize_conversation_with_function_calling()

    # Get initial response
    response = get_chat_completions_with_functions(conversation, use_functions=True)
    print("Assistant:", response.content)

    user_requirements = None
    top_3_laptops = None

    while True:
        user_input = input("\nUser: ")

        if user_input.lower() == "exit":
            break

        # Check moderation
        moderation = moderation_check(user_input)
        if moderation == 'Flagged':
            print("Sorry, this message has been flagged. Please restart your conversation.")
            break

        conversation.append({"role": "user", "content": user_input})

        if user_requirements is None:
            # Still gathering requirements
            response = get_chat_completions_with_functions(conversation, use_functions=True)

            # Check moderation on response
            if hasattr(response, 'content') and response.content:
                moderation = moderation_check(response.content)
                if moderation == 'Flagged':
                    print("Sorry, this message has been flagged. Please restart your conversation.")
                    break

            if response.function_call:
                # Function was called - extract requirements
                function_name = response.function_call.name
                function_args = json.loads(response.function_call.arguments)

                if function_name == "extract_user_requirements":
                    user_requirements = extract_user_requirements(**function_args)

                    if "error" in user_requirements:
                        print("Assistant:", user_requirements["error"])
                        continue

                    print("\nAssistant: Thank you for providing all the information!")
                    print("Requirements extracted:", user_requirements)
                    print("\nFetching laptop recommendations...")

                    # Get laptop recommendations
                    top_3_laptops = compare_laptops_with_user_enhanced(user_requirements)
                    validated_reco = recommendation_validation(top_3_laptops)

                    if not validated_reco:
                        print("Sorry, no laptops match your exact requirements. Let me find the best available options...")
                        # Could implement fallback logic here

                    # Generate recommendation response
                    conversation_reco = initialize_conv_reco(validated_reco)
                    conversation_reco.append({"role": "user", "content": f"User profile: {user_requirements}"})

                    recommendation_response = get_chat_completions(conversation_reco)
                    print("\nAssistant:", recommendation_response)

                    conversation = conversation_reco  # Switch to recommendation conversation
            else:
                # Continue conversation
                conversation.append({"role": "assistant", "content": response.content})
                print("\nAssistant:", response.content)
        else:
            # In recommendation phase
            conversation.append({"role": "user", "content": user_input})
            response = get_chat_completions(conversation)

            moderation = moderation_check(response)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            conversation.append({"role": "assistant", "content": response})
            print("\nAssistant:", response)

def compare_laptops_with_user_enhanced(user_requirements):
    """
    Enhanced version that works with function calling output
    """
    laptop_df = pd.read_csv('updated_laptop_enhanced.csv')

    # Convert the 'laptop_feature' column from string representation to dictionary
    laptop_df['laptop_feature'] = laptop_df['laptop_feature'].apply(ast.literal_eval)

    # Extract budget and filter
    budget = int(user_requirements.get('Budget', '0'))

    filtered_laptops = laptop_df.copy()
    filtered_laptops['Price'] = filtered_laptops['Price'].str.replace(',', '').astype(int)
    filtered_laptops = filtered_laptops[filtered_laptops['Price'] <= budget].copy()

    # Scoring logic (same as original)
    mappings = {'low': 0, 'medium': 1, 'high': 2}
    filtered_laptops['Score'] = 0

    for index, row in filtered_laptops.iterrows():
        laptop_values = row['laptop_feature']
        score = 0

        for key, user_value in user_requirements.items():
            if key == 'Budget':
                continue
            laptop_value = laptop_values.get(key, None)
            laptop_mapping = mappings.get(laptop_value, -1)
            user_mapping = mappings.get(user_value, -1)
            if laptop_mapping >= user_mapping:
                score += 1

        filtered_laptops.loc[index, 'Score'] = score

    # Get top 3
    top_laptops = filtered_laptops.drop('laptop_feature', axis=1)
    top_laptops = top_laptops.sort_values('Score', ascending=False).head(3)

    return top_laptops.to_json(orient='records')

In [60]:
process_laptop_dataset_enhanced()

Processing laptop dataset with enhanced function calling...
Processed 20 laptops with enhanced function calling


,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description,laptop_feature
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...,"{'GPU intensity': 'low', 'Display quality': 'm..."
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...,"{'GPU intensity': 'high', 'Display quality': '..."
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...,"{'GPU intensity': 'low', 'Display quality': 'm..."
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...,"{'GPU intensity': 'low', 'Display quality': 'l..."
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...,"{'GPU intensity': 'high', 'Display quality': '..."
5,Acer,Predator,i7,Intel,2.8 GHz,16GB,SSD,IPS,"17.3""",NVIDIA GTX,1920x1080,Windows 10,3.2 kg,Dual Cooling Fans,1 year,5 hours,"80,000",The Acer Predator is a powerhouse laptop desig...,"{'GPU intensity': 'medium', 'Display quality':..."
6,Microsoft,Surface Laptop,i5,Intel,1.6 GHz,8GB,SSD,PixelSense,"13.5""",Intel Iris Plus,2256x1504,Windows 11,1.3 kg,Touchscreen Display,1 year,10 hours,"90,000",The Microsoft Surface Laptop is a premium devi...,"{'GPU intensity': 'medium', 'Display quality':..."
7,Lenovo,ThinkPad,Ryzen 7,AMD,3.0 GHz,16GB,SSD,IPS,"14""",NVIDIA GTX,2560x1440,Linux,1.6 kg,Backlit Keyboard,3 years,6 hours,"60,000",The Lenovo ThinkPad is a powerful laptop desig...,"{'GPU intensity': 'medium', 'Display quality':..."
8,HP,Pavilion,i5,Intel,2.3 GHz,12GB,HDD,LCD,"15.6""",Intel UHD,1366x768,Windows 10,2.1 kg,B&O Audio,1 year,4 hours,"30,000",The HP Pavilion is a budget-friendly laptop th...,"{'GPU intensity': 'low', 'Display quality': 'l..."
9,ASUS,ROG Strix G,i7,Intel,2.9 GHz,16GB,SSD,IPS,"17.3""",NVIDIA RTX,1920x1080,Windows 10,2.9 kg,Aura Sync RGB Keyboard,2 years,5 hours,"85,000",The ASUS ROG Strix G is a high-performance gam...,"{'GPU intensity': 'high', 'Display quality': '..."


In [63]:
dialogue_mgmt_system_enhanced()

Assistant: Hello! I'm here to help you find the perfect laptop. Could you please tell me what you'll be primarily using your laptop for? For example, is it for gaming, work, video editing, or something else?

User: gaming

Assistant: That's great! Gaming laptops have specific requirements to ensure the best experience. 

1. Do you prefer playing high-end games that require a powerful GPU, or are you more into casual gaming? 
2. Also, how important is the display quality to you? Would you prefer a high-resolution display with color accuracy or is a standard display fine for you?

User: high gpu, high processing speed, medium multitasking, medium display

Assistant: Perfect, that's very helpful. 

Now, let's talk about portability. Do you need a laptop that's easy to carry around, or will it mostly stay in one place? 

And finally, could you please share your budget range for this purchase?

User: portability is not important, budget is 125000

Assistant: Thank you for providing all the 

### **Key Benefits of the New Approach:**

**Simplified Logic:**
- No more complex prompt engineering for data extraction
- Automatic validation through schema constraints
- Guaranteed structured output format

**Better Accuracy:**
- Function calling eliminates parsing errors
- Schema validation ensures correct data types
- Consistent output format every time

**Cleaner Code:**
- Separation of conversation logic from data extraction
- Less complex prompts to maintain
- Easier debugging and testing